Neste exemplo, vamos demonstrar as contas como são feitas para a LSTM, ao invés de RNN's como fizemos no [exemplo anterior](./12-simple-rnn.qmd).


In [ ]:
import numpy as np
from tensorflow import keras

Primeiro, vamos gerar os dados novamente:

In [ ]:
n = 10000
l = 10

cresc = np.random.randint(0, 2, size=(n,))
x = np.empty((n, l), dtype=np.float32)
for i, cr in enumerate(cresc):
    tmp = np.random.uniform(size=(l,))
    if cr == 1:
        x[i, :] = tmp[np.argsort(tmp)]
    else:
        x[i, :] = tmp[np.argsort(-tmp)]
x = x.reshape((n, l, 1))

Agora vamos definir o modelo no Keras. Não vamos nos preocupar com os
parâmetros nem nada ainda. A seguir vamos mostrar exatamente as contas que o Keras está fazendo por trás.

In [ ]:
input = keras.layers.Input(shape=(l, 1))
output = keras.layers.LSTM(units=1, activation="tanh", use_bias=False, recurrent_activation="sigmoid")(input)
output = keras.layers.Activation("sigmoid")(output)
model = keras.Model(inputs=input, outputs=output)

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics="accuracy")

Agora ajustamos o modelo:

In [ ]:
#| layout-ncol: 1
#| column: screen-right
model.fit(x=x, y=cresc, epochs=10, batch_size=32, verbose=2)

Agora vamos ver a conta linha por linha que é feita pelo Keras para cada
observação. Definimos as funções que vamos usar:

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Selecionamos uma observação:

In [ ]:
x_ = x[0]

Obtemos os pesos estimados pelo Keras:

In [ ]:
w = model.get_weights()

Inicializamos os estados `s` e `c` e dentro do loop escrevemos a regra de atualização dos estados:

In [ ]:
s = 0
c = 0


for t in range(l):
    f = sigmoid(w[1][0, 1] * s + w[0][0, 1] * x_[t])
    i = sigmoid(w[1][0, 0] * s + w[0][0, 0] * x_[t])
    c_hat = np.tanh(w[1][0, 2] * s + w[0][0, 2] * x_[t])
    c = f * c + i * c_hat
    o = sigmoid(s * w[1][0, 3] + w[0][0, 3] * x_[t])
    s = o * np.tanh(c)

Emfim, aplicamos o sigmoid no estado final:

In [ ]:
sigmoid(s)

O valor predito pelo keras para essa observação seria:

In [ ]:
model.predict(x)[0]